In [14]:
from neighbor_server import *
from social_graph import *
from shortest_paths import *

In [15]:
sg = SocialGraph("train")

Initializing check ins
Loading from Training Pickle File
Initializing network


In [16]:
from __future__ import print_function

In [4]:
'''
when you add neighbors into the graph
their g(source) = dist of last check in from source
and their h(n) = dist of last check in from end

g is cost of geographic movement from source
h becomes hueristic to choose the closest neighbor to goal

to throttle the API,
pick the top k nodes at each step that have f = g + h the minimum
rather than putting all nodes in the priority queue

otherwise put all the nodes in the queue, hoping for hueristic to take care of the problem
also maintain the got_neighbors_for set so that multiple test cases can use the same graph
'''

'\nwhen you add neighbors into the graph\ntheir g(source) = dist of last check in from source\nand their h(n) = dist of last check in from end\n\ng is cost of geographic movement from source\nh becomes hueristic to choose the closest neighbor to goal\n\nto throttle the API,\npick the top k nodes at each step that have f = g + h the minimum\nrather than putting all nodes in the priority queue\n\notherwise put all the nodes in the queue, hoping for hueristic to take care of the problem\nalso maintain the got_neighbors_for set so that multiple test cases can use the same graph\n'

In [26]:
import sys

In [4]:
shortest_path, path_length, num_queries, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.unweighted, h_fn=dists.l2)

Getting neighbors for: 5555
Getting neighbors for: 648
Getting neighbors for: 12614
Getting neighbors for: 647
Getting neighbors for: 649
Getting neighbors for: 9321
Getting neighbors for: 18163
Getting neighbors for: 36559
Getting neighbors for: 280
Getting neighbors for: 9327
Getting neighbors for: 40345
Getting neighbors for: 9324
Getting neighbors for: 9322
Getting neighbors for: 9323
Getting neighbors for: 32515
Getting neighbors for: 7576
Getting neighbors for: 11576
Getting neighbors for: 41857
Getting neighbors for: 15665
Getting neighbors for: 13711
Getting neighbors for: 964
Getting neighbors for: 22591
Getting neighbors for: 21373
Getting neighbors for: 2202
Getting neighbors for: 11287
Getting neighbors for: 13394
Getting neighbors for: 15947
Getting neighbors for: 37713
Getting neighbors for: 6883
Getting neighbors for: 20287
Getting neighbors for: 25511
Getting neighbors for: 17599
Getting neighbors for: 11592
Getting neighbors for: 40568
Getting neighbors for: 11290
Gett

In [9]:
shortest_path

[5555, 648, 30447, 2222]

In [ ]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.unweighted, h_fn=dists.l2)

In [13]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 939, 401, set(), "train", g_fn=dists.unweighted, h_fn=dists.manhattan)

Destination Reached


In [14]:
shortest_path

[939, 5294, 27493, 10278, 401]

In [15]:
path_length

4

In [18]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.manhattan, h_fn=dists.manhattan)

Destination Reached


In [19]:
shortest_path

[5555, 8, 5187, 19390, 2222]

In [10]:
# path length
task1_combs = [(dists.unweighted, dists.manhattan), (dists.unweighted, dists.l2)]
# geographic length
task2_combs = [(dists.manhattan, dists.manhattan), (dists.l2, dists.l2)]

In [13]:
source = 5555
dest = 2222
for g_fn, h_fn in task1_combs:
    shortest_path, path_length, num_queries, got_neighbors_for = astar_distance_based(
        sg, source, dest, set(), "train", 
        g_fn, h_fn
    )
    print(shortest_path)

Getting neighbors for: 5555
Getting neighbors for: 8
Getting neighbors for: 221
Getting neighbors for: 839
Getting neighbors for: 3662
Getting neighbors for: 4488
Getting neighbors for: 13788
Getting neighbors for: 15157
Getting neighbors for: 31378
Getting neighbors for: 43963
Getting neighbors for: 37
Getting neighbors for: 21559
Getting neighbors for: 186
Getting neighbors for: 260
Getting neighbors for: 307
Getting neighbors for: 582
Getting neighbors for: 4481
Getting neighbors for: 974
Getting neighbors for: 983
Getting neighbors for: 1863
Getting neighbors for: 2639
Getting neighbors for: 4397
Getting neighbors for: 4398
Getting neighbors for: 4478
Getting neighbors for: 4479
Getting neighbors for: 4896
Getting neighbors for: 5003
Getting neighbors for: 5185
Getting neighbors for: 5187
Getting neighbors for: 5260
Getting neighbors for: 5262
Getting neighbors for: 6468
Getting neighbors for: 7561
Getting neighbors for: 7566
Getting neighbors for: 4019
Getting neighbors for: 8245


ValueError: need more than 3 values to unpack